<a href="https://www.kaggle.com/code/mirkocaroleo/regression-with-an-insurance-dataset?scriptVersionId=215440867" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e12/sample_submission.csv
/kaggle/input/playground-series-s4e12/train.csv
/kaggle/input/playground-series-s4e12/test.csv


In [2]:
import os
print(os.listdir('/kaggle/input'))

['playground-series-s4e12']


In [3]:
import pandas as InterruptedError

train_df = pd.read_csv('/kaggle/input/playground-series-s4e12/train.csv')
test_df = pd.read_csv('/kaggle/input/playground-series-s4e12/test.csv')

train_df.head()

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
0,0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,...,2.0,17.0,372.0,5.0,2023-12-23 15:21:39.134960,Poor,No,Weekly,House,2869.0
1,1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,...,1.0,12.0,694.0,2.0,2023-06-12 15:21:39.111551,Average,Yes,Monthly,House,1483.0
2,2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,...,1.0,14.0,NaN,3.0,2023-09-30 15:21:39.221386,Good,Yes,Weekly,House,567.0
3,3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,...,1.0,0.0,367.0,1.0,2024-06-12 15:21:39.226954,Poor,Yes,Daily,Apartment,765.0
4,4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,...,0.0,8.0,598.0,4.0,2021-12-01 15:21:39.252145,Poor,Yes,Weekly,House,2022.0


In [4]:
print(train_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200000 entries, 0 to 1199999
Data columns (total 21 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   id                    1200000 non-null  int64  
 1   Age                   1181295 non-null  float64
 2   Gender                1200000 non-null  object 
 3   Annual Income         1155051 non-null  float64
 4   Marital Status        1181471 non-null  object 
 5   Number of Dependents  1090328 non-null  float64
 6   Education Level       1200000 non-null  object 
 7   Occupation            841925 non-null   object 
 8   Health Score          1125924 non-null  float64
 9   Location              1200000 non-null  object 
 10  Policy Type           1200000 non-null  object 
 11  Previous Claims       835971 non-null   float64
 12  Vehicle Age           1199994 non-null  float64
 13  Credit Score          1062118 non-null  float64
 14  Insurance Duration    1199999 non-

In [5]:
print(train_df.isnull().sum())

id                           0
Age                      18705
Gender                       0
Annual Income            44949
Marital Status           18529
Number of Dependents    109672
Education Level              0
Occupation              358075
Health Score             74076
Location                     0
Policy Type                  0
Previous Claims         364029
Vehicle Age                  6
Credit Score            137882
Insurance Duration           1
Policy Start Date            0
Customer Feedback        77824
Smoking Status               0
Exercise Frequency           0
Property Type                0
Premium Amount               0
dtype: int64


In [6]:
print(train_df.shape)

(1200000, 21)


In [7]:
# Unique values for categorical columns
print(train_df['Marital Status'].value_counts())
print(train_df['Occupation'].value_counts())
print(train_df['Education Level'].value_counts())
print(train_df['Policy Type'].value_counts())
print(train_df['Exercise Frequency'].value_counts())
print(train_df['Smoking Status'].value_counts())

Marital Status
Single      395391
Married     394316
Divorced    391764
Name: count, dtype: int64
Occupation
Employed         282750
Self-Employed    282645
Unemployed       276530
Name: count, dtype: int64
Education Level
Master's       303818
PhD            303507
Bachelor's     303234
High School    289441
Name: count, dtype: int64
Policy Type
Premium          401846
Comprehensive    399600
Basic            398554
Name: count, dtype: int64
Exercise Frequency
Weekly     306179
Monthly    299830
Rarely     299420
Daily      294571
Name: count, dtype: int64
Smoking Status
Yes    601873
No     598127
Name: count, dtype: int64


In [8]:
print(train_df.dtypes)

id                        int64
Age                     float64
Gender                   object
Annual Income           float64
Marital Status           object
Number of Dependents    float64
Education Level          object
Occupation               object
Health Score            float64
Location                 object
Policy Type              object
Previous Claims         float64
Vehicle Age             float64
Credit Score            float64
Insurance Duration      float64
Policy Start Date        object
Customer Feedback        object
Smoking Status           object
Exercise Frequency       object
Property Type            object
Premium Amount          float64
dtype: object


Now that we have a better overview of the dataframe we need to understand which columns are going to affect the Premium Amount column, our goal column.

To do so we can use the correlation matrix.

Before implementing the correlation matrix we need however to encode the categorical columns, as well as deal with missing values, and transform non-numerical data into something our correlation matrix and model will be able to interpret.

how to deal with NaN values for different columns:

Age, assign AVG to NaN, Annual Income - assign Mean, Marital Status - assign Single, Number of Dependents - assign 0, Occupation - assign Unemployed, Health Score - assign Mean, Previous Claims - assign 0, Vehicle Age - assign 0, Credit Score - assign mean, Insurance Duration - assign MEAN, Customer Feedback - drop column as not influencial on the premium amount.

In [9]:
#Dealing with missing data
train_df['Age'] = train_df['Age'].fillna(train_df['Age'].mean())
train_df['Annual Income'] = train_df['Annual Income'].fillna(train_df['Annual Income'].mean())
train_df['Marital Status'] = train_df['Marital Status'].fillna('Single')
train_df['Number of Dependents'] = train_df['Number of Dependents'].fillna(0)
train_df['Occupation'] = train_df['Occupation'].fillna('Unemployed')
train_df['Health Score'] = train_df['Health Score'].fillna(train_df['Health Score'].mean())
train_df['Previous Claims'] = train_df['Previous Claims'].fillna(0)
train_df['Vehicle Age'] = train_df['Vehicle Age'].fillna(0)
train_df['Credit Score'] = train_df['Credit Score'].fillna(train_df['Credit Score'].mean())
train_df['Insurance Duration'] = train_df['Insurance Duration'].fillna(train_df['Insurance Duration'].mean())

In [10]:
#dropping customer feedback
train_df = train_df.drop(columns=['Customer Feedback'])
print(train_df.head())

   id   Age  Gender  Annual Income Marital Status  Number of Dependents  \
0   0  19.0  Female        10049.0        Married                   1.0   
1   1  39.0  Female        31678.0       Divorced                   3.0   
2   2  23.0    Male        25602.0       Divorced                   3.0   
3   3  21.0    Male       141855.0        Married                   2.0   
4   4  21.0    Male        39651.0         Single                   1.0   

  Education Level     Occupation  Health Score  Location    Policy Type  \
0      Bachelor's  Self-Employed     22.598761     Urban        Premium   
1        Master's     Unemployed     15.569731     Rural  Comprehensive   
2     High School  Self-Employed     47.177549  Suburban        Premium   
3      Bachelor's     Unemployed     10.938144     Rural          Basic   
4      Bachelor's  Self-Employed     20.376094     Rural        Premium   

   Previous Claims  Vehicle Age  Credit Score  Insurance Duration  \
0              2.0         17

In [11]:
print(train_df.isnull().sum())

id                      0
Age                     0
Gender                  0
Annual Income           0
Marital Status          0
Number of Dependents    0
Education Level         0
Occupation              0
Health Score            0
Location                0
Policy Type             0
Previous Claims         0
Vehicle Age             0
Credit Score            0
Insurance Duration      0
Policy Start Date       0
Smoking Status          0
Exercise Frequency      0
Property Type           0
Premium Amount          0
dtype: int64


Now that we have dealt with missing values we must modify the date columns to timeframe for better readibility of the model.

Next steps for me to carry on:

timeframe columns

encoding

correlation matrix

model training

In [12]:
#converting the date to datetime
train_df['Policy Start Date'] = pd.to_datetime(train_df['Policy Start Date'])

#confirm the changes
print(train_df['Policy Start Date'].dtype)

datetime64[ns]


In [13]:
#encoding the categorical columns Marital Status and Occupation
encoded_df = pd.get_dummies(train_df, columns=['Gender','Marital Status', 'Occupation','Location','Property Type','Smoking Status'], drop_first = True)

print(encoded_df.head())

   id   Age  Annual Income  Number of Dependents Education Level  \
0   0  19.0        10049.0                   1.0      Bachelor's   
1   1  39.0        31678.0                   3.0        Master's   
2   2  23.0        25602.0                   3.0     High School   
3   3  21.0       141855.0                   2.0      Bachelor's   
4   4  21.0        39651.0                   1.0      Bachelor's   

   Health Score    Policy Type  Previous Claims  Vehicle Age  Credit Score  \
0     22.598761        Premium              2.0         17.0     372.00000   
1     15.569731  Comprehensive              1.0         12.0     694.00000   
2     47.177549        Premium              1.0         14.0     592.92435   
3     10.938144          Basic              1.0          0.0     367.00000   
4     20.376094        Premium              0.0          8.0     598.00000   

   ...  Gender_Male Marital Status_Married Marital Status_Single  \
0  ...        False                   True            

In [14]:
#encoding the ordinal categorical values for Education Level and Exercise Frequency

education_mapping = {'High School': 0, 'Bachelor\'s': 1, 'Master\'s': 2, 'PhD': 3}
encoded_df['Education Level'] = train_df['Education Level'].map(education_mapping)

exercise_mapping = {'Rarely': 0, 'Monthly': 1, 'Weekly': 2, 'Daily': 3}
encoded_df['Exercise Frequency'] = train_df['Exercise Frequency'].map(exercise_mapping)

policy_mapping = {'Basic': 0, 'Comprehensive': 1, 'Premium': 2}
encoded_df['Policy Type'] = train_df['Policy Type'].map(policy_mapping)

print(encoded_df[['Education Level','Exercise Frequency', 'Policy Type']].head())

   Education Level  Exercise Frequency  Policy Type
0                1                   2            2
1                2                   1            1
2                0                   2            2
3                1                   3            0
4                1                   2            2


Now that we have encoded the df we are going to check that all of the columns are numerical.

In [15]:
print(encoded_df.dtypes)

id                                   int64
Age                                float64
Annual Income                      float64
Number of Dependents               float64
Education Level                      int64
Health Score                       float64
Policy Type                          int64
Previous Claims                    float64
Vehicle Age                        float64
Credit Score                       float64
Insurance Duration                 float64
Policy Start Date           datetime64[ns]
Exercise Frequency                   int64
Premium Amount                     float64
Gender_Male                           bool
Marital Status_Married                bool
Marital Status_Single                 bool
Occupation_Self-Employed              bool
Occupation_Unemployed                 bool
Location_Suburban                     bool
Location_Urban                        bool
Property Type_Condo                   bool
Property Type_House                   bool
Smoking Sta

Datetime is going to be a problem for the correlation matrix. For this reason we are going to calculate the age of the account with a datetime difference function.

In [16]:
#transforming the start date column into an age account column
import datetime
encoded_df['Policy Duration (days)'] = (datetime.datetime.now() - encoded_df['Policy Start Date']).dt.days

encoded_df = encoded_df.drop(columns=['Policy Start Date'])

print(encoded_df['Policy Duration (days)'].head())


0     372
1     566
2     456
3     200
4    1124
Name: Policy Duration (days), dtype: int64


In [17]:
print(encoded_df.head())

   id   Age  Annual Income  Number of Dependents  Education Level  \
0   0  19.0        10049.0                   1.0                1   
1   1  39.0        31678.0                   3.0                2   
2   2  23.0        25602.0                   3.0                0   
3   3  21.0       141855.0                   2.0                1   
4   4  21.0        39651.0                   1.0                1   

   Health Score  Policy Type  Previous Claims  Vehicle Age  Credit Score  ...  \
0     22.598761            2              2.0         17.0     372.00000  ...   
1     15.569731            1              1.0         12.0     694.00000  ...   
2     47.177549            2              1.0         14.0     592.92435  ...   
3     10.938144            0              1.0          0.0     367.00000  ...   
4     20.376094            2              0.0          8.0     598.00000  ...   

   Marital Status_Married  Marital Status_Single  Occupation_Self-Employed  \
0                   

Now we can finally proceed to the correlation matrix.

In [18]:
correlation_matrix = encoded_df.corr()
print(correlation_matrix['Premium Amount'].sort_values(ascending = False))

Premium Amount              1.000000
Previous Claims             0.043885
Health Score                0.014326
Policy Duration (days)      0.010111
Marital Status_Single       0.003682
Occupation_Self-Employed    0.002608
Location_Urban              0.000952
Vehicle Age                 0.000390
Exercise Frequency          0.000378
Smoking Status_Yes          0.000163
Gender_Male                 0.000161
Insurance Duration         -0.000028
Location_Suburban          -0.000068
id                         -0.000292
Property Type_Condo        -0.000508
Property Type_House        -0.000804
Policy Type                -0.000884
Education Level            -0.001654
Marital Status_Married     -0.002184
Age                        -0.002411
Number of Dependents       -0.004306
Occupation_Unemployed      -0.004867
Annual Income              -0.012091
Credit Score               -0.024438
Name: Premium Amount, dtype: float64


Correlations seem to be weak. This is somehow expected when working with insurance policies as the Premium Amount is often a result of complex calculations with multiple models.

Since the competition requests us to use Linear Regression but the correlation is weak there are a few extra steps we need to complete before proceeding and training the model.

Feature Engineering:

Weak correlations indicate that raw features might not capture relationships effectively. Transformations and interaction terms can help.

Multicollinearity:
One-hot encoding increases the number of features, which could lead to multicollinearity (high correlation between features).

Scaling Continuous Features:
Features like Age, Annual Income, and Policy Duration (days) might have very different scales, which could make the model less stable.

Steps to Ensure a Strong Linear Regression Model
1. Prepare the Data
Before training the model:

Handle Feature Scaling:

Standardize or normalize numerical features (Age, Annual Income, Credit Score, etc.) to ensure they have comparable scales.

In [19]:
from sklearn.preprocessing import StandardScaler

# Select numerical columns
numerical_columns = ['Age', 'Annual Income', 'Number of Dependents', 'Health Score',
                     'Policy Duration (days)', 'Credit Score', 'Insurance Duration']

# Initialize scaler and apply to numerical columns
scaler = StandardScaler()
encoded_df[numerical_columns] = scaler.fit_transform(encoded_df[numerical_columns])

Interaction Terms (Optional):

Introduce interaction terms to capture relationships between features:

In [20]:
# Example: Interaction between Age and Health Score
encoded_df['Age_Health_Interaction'] = encoded_df['Age'] * encoded_df['Health Score']

Check Multicollinearity:

Use variance inflation factor (VIF) to ensure features are not highly collinear:

In [21]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Calculate VIF for each feature
X = encoded_df.drop(columns=['Premium Amount'])  # Exclude target

#Let's first make sure we have all numerical columns. If some columns are different than int64 we need to transform them.
print(X.dtypes)

id                            int64
Age                         float64
Annual Income               float64
Number of Dependents        float64
Education Level               int64
Health Score                float64
Policy Type                   int64
Previous Claims             float64
Vehicle Age                 float64
Credit Score                float64
Insurance Duration          float64
Exercise Frequency            int64
Gender_Male                    bool
Marital Status_Married         bool
Marital Status_Single          bool
Occupation_Self-Employed       bool
Occupation_Unemployed          bool
Location_Suburban              bool
Location_Urban                 bool
Property Type_Condo            bool
Property Type_House            bool
Smoking Status_Yes             bool
Policy Duration (days)      float64
Age_Health_Interaction      float64
dtype: object


In [22]:
#Convert bool Columns to Integers

X = X.astype({col: 'int64' for col in X.select_dtypes(include='bool').columns})


In [23]:
#check for infinite values

print(np.isfinite(X).all().all())  # Check for infinite values

True


In [24]:
#fixing infite values

X = X.replace([np.inf, -np.inf], np.nan)  # Replace inf with NaN
X = X.dropna()  # Drop rows with NaN

In [25]:
print(X.isnull().sum())  # Should return 0 for all columns if no NaN values remain

id                          0
Age                         0
Annual Income               0
Number of Dependents        0
Education Level             0
Health Score                0
Policy Type                 0
Previous Claims             0
Vehicle Age                 0
Credit Score                0
Insurance Duration          0
Exercise Frequency          0
Gender_Male                 0
Marital Status_Married      0
Marital Status_Single       0
Occupation_Self-Employed    0
Occupation_Unemployed       0
Location_Suburban           0
Location_Urban              0
Property Type_Condo         0
Property Type_House         0
Smoking Status_Yes          0
Policy Duration (days)      0
Age_Health_Interaction      0
dtype: int64


We can now calculate the VIF once again.

In [26]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif_data = pd.DataFrame()
vif_data['feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif_data)

                     feature       VIF
0                         id  3.492832
1                        Age  1.000043
2              Annual Income  1.038919
3       Number of Dependents  1.000059
4            Education Level  2.639241
5               Health Score  1.000929
6                Policy Type  2.347725
7            Previous Claims  1.518256
8                Vehicle Age  3.312255
9               Credit Score  1.037867
10        Insurance Duration  1.000028
11        Exercise Frequency  2.593286
12               Gender_Male  1.923969
13    Marital Status_Married  1.878871
14     Marital Status_Single  1.920924
15  Occupation_Self-Employed  1.823591
16     Occupation_Unemployed  2.845407
17         Location_Suburban  1.876017
18            Location_Urban  1.867897
19       Property Type_Condo  1.873662
20       Property Type_House  1.872759
21        Smoking Status_Yes  1.921864
22    Policy Duration (days)  1.000202
23    Age_Health_Interaction  1.000038


The VIF results are acceptable.absabs

We can either train the model with this dataset or deep dive even further to try and remove outliers from the df to reduce overfitting and noise.

In [27]:
#Recursive Feature Elimination (RFE) to try and identify the noisy features 
y = encoded_df['Premium Amount']
X = encoded_df.drop(columns=['Premium Amount'])

from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

model = LinearRegression()
rfe = RFE(model, n_features_to_select=15)  # Select top 15 features
rfe.fit(X, y)

print("Feature Ranking:", rfe.ranking_)  # Features ranked 1 are most important
print("Selected Features:", X.columns[rfe.support_])


Feature Ranking: [10  1  1  1  1  1  2  1  9  1  8  7  6  3  1  1  1  1  1  1  1  5  1  4]
Selected Features: Index(['Age', 'Annual Income', 'Number of Dependents', 'Education Level',
       'Health Score', 'Previous Claims', 'Credit Score',
       'Marital Status_Single', 'Occupation_Self-Employed',
       'Occupation_Unemployed', 'Location_Suburban', 'Location_Urban',
       'Property Type_Condo', 'Property Type_House', 'Policy Duration (days)'],
      dtype='object')


In [28]:
X_selected = X.loc[:, rfe.support_]

Before fitting the model we need to transform the test_df the same way we did with the train_df.

In [29]:
test_df.head()

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type
0,1200000,28.0,Female,2310.0,NaN,4.0,Bachelor's,Self-Employed,7.657981,Rural,Basic,NaN,19.0,NaN,1.0,2023-06-04 15:21:39.245086,Poor,Yes,Weekly,House
1,1200001,31.0,Female,126031.0,Married,2.0,Master's,Self-Employed,13.381379,Suburban,Premium,NaN,14.0,372.0,8.0,2024-04-22 15:21:39.224915,Good,Yes,Rarely,Apartment
2,1200002,47.0,Female,17092.0,Divorced,0.0,PhD,Unemployed,24.354527,Urban,Comprehensive,NaN,16.0,819.0,9.0,2023-04-05 15:21:39.134960,Average,Yes,Monthly,Condo
3,1200003,28.0,Female,30424.0,Divorced,3.0,PhD,Self-Employed,5.136225,Suburban,Comprehensive,1.0,3.0,770.0,5.0,2023-10-25 15:21:39.134960,Poor,Yes,Daily,House
4,1200004,24.0,Male,10863.0,Divorced,2.0,High School,Unemployed,11.844155,Suburban,Premium,NaN,14.0,755.0,7.0,2021-11-26 15:21:39.259788,Average,No,Weekly,House


In [30]:
print(test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800000 entries, 0 to 799999
Data columns (total 20 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    800000 non-null  int64  
 1   Age                   787511 non-null  float64
 2   Gender                800000 non-null  object 
 3   Annual Income         770140 non-null  float64
 4   Marital Status        787664 non-null  object 
 5   Number of Dependents  726870 non-null  float64
 6   Education Level       800000 non-null  object 
 7   Occupation            560875 non-null  object 
 8   Health Score          750551 non-null  float64
 9   Location              800000 non-null  object 
 10  Policy Type           800000 non-null  object 
 11  Previous Claims       557198 non-null  float64
 12  Vehicle Age           799997 non-null  float64
 13  Credit Score          708549 non-null  float64
 14  Insurance Duration    799998 non-null  float64
 15  

In [31]:
print(test_df.isnull().sum())

id                           0
Age                      12489
Gender                       0
Annual Income            29860
Marital Status           12336
Number of Dependents     73130
Education Level              0
Occupation              239125
Health Score             49449
Location                     0
Policy Type                  0
Previous Claims         242802
Vehicle Age                  3
Credit Score             91451
Insurance Duration           2
Policy Start Date            0
Customer Feedback        52276
Smoking Status               0
Exercise Frequency           0
Property Type                0
dtype: int64


In [32]:
print(test_df.dtypes)

id                        int64
Age                     float64
Gender                   object
Annual Income           float64
Marital Status           object
Number of Dependents    float64
Education Level          object
Occupation               object
Health Score            float64
Location                 object
Policy Type              object
Previous Claims         float64
Vehicle Age             float64
Credit Score            float64
Insurance Duration      float64
Policy Start Date        object
Customer Feedback        object
Smoking Status           object
Exercise Frequency       object
Property Type            object
dtype: object


In [33]:
#Dealing with missing data
test_df['Age'] = test_df['Age'].fillna(test_df['Age'].mean())
test_df['Annual Income'] = test_df['Annual Income'].fillna(test_df['Annual Income'].mean())
test_df['Marital Status'] = test_df['Marital Status'].fillna('Single')
test_df['Number of Dependents'] = test_df['Number of Dependents'].fillna(0)
test_df['Occupation'] = test_df['Occupation'].fillna('Unemployed')
test_df['Health Score'] = test_df['Health Score'].fillna(test_df['Health Score'].mean())
test_df['Previous Claims'] = test_df['Previous Claims'].fillna(0)
test_df['Vehicle Age'] = test_df['Vehicle Age'].fillna(0)
test_df['Credit Score'] = test_df['Credit Score'].fillna(test_df['Credit Score'].mean())
test_df['Insurance Duration'] = test_df['Insurance Duration'].fillna(test_df['Insurance Duration'].mean())


In [34]:
#dropping customer feedback
test_df = test_df.drop(columns=['Customer Feedback'])
print(test_df.head())

        id   Age  Gender  Annual Income Marital Status  Number of Dependents  \
0  1200000  28.0  Female         2310.0         Single                   4.0   
1  1200001  31.0  Female       126031.0        Married                   2.0   
2  1200002  47.0  Female        17092.0       Divorced                   0.0   
3  1200003  28.0  Female        30424.0       Divorced                   3.0   
4  1200004  24.0    Male        10863.0       Divorced                   2.0   

  Education Level     Occupation  Health Score  Location    Policy Type  \
0      Bachelor's  Self-Employed      7.657981     Rural          Basic   
1        Master's  Self-Employed     13.381379  Suburban        Premium   
2             PhD     Unemployed     24.354527     Urban  Comprehensive   
3             PhD  Self-Employed      5.136225  Suburban  Comprehensive   
4     High School     Unemployed     11.844155  Suburban        Premium   

   Previous Claims  Vehicle Age  Credit Score  Insurance Duration  \

In [35]:
#converting the date to datetime
test_df['Policy Start Date'] = pd.to_datetime(test_df['Policy Start Date'])


In [36]:
#confirm the changes
print(train_df['Policy Start Date'].dtype)

datetime64[ns]


In [37]:
#encoding the categorical columns Marital Status and Occupation
test_encoded_df = pd.get_dummies(test_df, columns=['Gender','Marital Status', 'Occupation','Location','Property Type','Smoking Status'], drop_first = True)

print(test_encoded_df.head())


        id   Age  Annual Income  Number of Dependents Education Level  \
0  1200000  28.0         2310.0                   4.0      Bachelor's   
1  1200001  31.0       126031.0                   2.0        Master's   
2  1200002  47.0        17092.0                   0.0             PhD   
3  1200003  28.0        30424.0                   3.0             PhD   
4  1200004  24.0        10863.0                   2.0     High School   

   Health Score    Policy Type  Previous Claims  Vehicle Age  Credit Score  \
0      7.657981          Basic              0.0         19.0    592.904749   
1     13.381379        Premium              0.0         14.0    372.000000   
2     24.354527  Comprehensive              0.0         16.0    819.000000   
3      5.136225  Comprehensive              1.0          3.0    770.000000   
4     11.844155        Premium              0.0         14.0    755.000000   

   ...  Gender_Male Marital Status_Married Marital Status_Single  \
0  ...        False     

In [38]:
#encoding the ordinal categorical values for Education Level and Exercise Frequency

education_mapping = {'High School': 0, 'Bachelor\'s': 1, 'Master\'s': 2, 'PhD': 3}
test_encoded_df['Education Level'] = test_df['Education Level'].map(education_mapping)

exercise_mapping = {'Rarely': 0, 'Monthly': 1, 'Weekly': 2, 'Daily': 3}
test_encoded_df['Exercise Frequency'] = test_df['Exercise Frequency'].map(exercise_mapping)

policy_mapping = {'Basic': 0, 'Comprehensive': 1, 'Premium': 2}
test_encoded_df['Policy Type'] = test_df['Policy Type'].map(policy_mapping)

print(test_encoded_df[['Education Level','Exercise Frequency', 'Policy Type']].head())

print(test_encoded_df.dtypes)


   Education Level  Exercise Frequency  Policy Type
0                1                   2            0
1                2                   0            2
2                3                   1            1
3                3                   3            1
4                0                   2            2
id                                   int64
Age                                float64
Annual Income                      float64
Number of Dependents               float64
Education Level                      int64
Health Score                       float64
Policy Type                          int64
Previous Claims                    float64
Vehicle Age                        float64
Credit Score                       float64
Insurance Duration                 float64
Policy Start Date           datetime64[ns]
Exercise Frequency                   int64
Gender_Male                           bool
Marital Status_Married                bool
Marital Status_Single                 bool


In [39]:
#transforming the start date column into an age account column
import datetime
test_encoded_df['Policy Duration (days)'] = (datetime.datetime.now() - test_encoded_df['Policy Start Date']).dt.days

test_encoded_df = test_encoded_df.drop(columns=['Policy Start Date'])

print(test_encoded_df['Policy Duration (days)'].head())

print(test_encoded_df.head())


0     574
1     251
2     634
3     431
4    1129
Name: Policy Duration (days), dtype: int64
        id   Age  Annual Income  Number of Dependents  Education Level  \
0  1200000  28.0         2310.0                   4.0                1   
1  1200001  31.0       126031.0                   2.0                2   
2  1200002  47.0        17092.0                   0.0                3   
3  1200003  28.0        30424.0                   3.0                3   
4  1200004  24.0        10863.0                   2.0                0   

   Health Score  Policy Type  Previous Claims  Vehicle Age  Credit Score  ...  \
0      7.657981            0              0.0         19.0    592.904749  ...   
1     13.381379            2              0.0         14.0    372.000000  ...   
2     24.354527            1              0.0         16.0    819.000000  ...   
3      5.136225            1              1.0          3.0    770.000000  ...   
4     11.844155            2              0.0         14.

In [40]:
from sklearn.preprocessing import StandardScaler

# Select numerical columns
numerical_columns = ['Age', 'Annual Income', 'Number of Dependents', 'Health Score',
                     'Policy Duration (days)', 'Credit Score', 'Insurance Duration']

# Initialize scaler and apply to numerical columns
scaler = StandardScaler()
test_encoded_df[numerical_columns] = scaler.fit_transform(test_encoded_df[numerical_columns])


In [41]:
# Example: Interaction between Age and Health Score
test_encoded_df['Age_Health_Interaction'] = test_encoded_df['Age'] * test_encoded_df['Health Score']


In [42]:
# Separate features and target in the training data
X_train = encoded_df.drop(columns=['Premium Amount'])  # Exclude the target column from features
y_train = encoded_df['Premium Amount']  # Target column

X_test = test_encoded_df

print("Columns in X_train:")
print(X_train.columns)

print("\nColumns in X_test:")
print(X_test.columns)


Columns in X_train:
Index(['id', 'Age', 'Annual Income', 'Number of Dependents', 'Education Level',
       'Health Score', 'Policy Type', 'Previous Claims', 'Vehicle Age',
       'Credit Score', 'Insurance Duration', 'Exercise Frequency',
       'Gender_Male', 'Marital Status_Married', 'Marital Status_Single',
       'Occupation_Self-Employed', 'Occupation_Unemployed',
       'Location_Suburban', 'Location_Urban', 'Property Type_Condo',
       'Property Type_House', 'Smoking Status_Yes', 'Policy Duration (days)',
       'Age_Health_Interaction'],
      dtype='object')

Columns in X_test:
Index(['id', 'Age', 'Annual Income', 'Number of Dependents', 'Education Level',
       'Health Score', 'Policy Type', 'Previous Claims', 'Vehicle Age',
       'Credit Score', 'Insurance Duration', 'Exercise Frequency',
       'Gender_Male', 'Marital Status_Married', 'Marital Status_Single',
       'Occupation_Self-Employed', 'Occupation_Unemployed',
       'Location_Suburban', 'Location_Urban', 'Proper

In [43]:
print(f"Number of columns in X_train: {X_train.shape[1]}")
print(f"Number of columns in X_test: {X_test.shape[1]}")

Number of columns in X_train: 24
Number of columns in X_test: 24


In [44]:
# Drop 'id' from both X_train and X_test
X_train = encoded_df.drop(columns=['id', 'Premium Amount'])  # Drop 'id' and the target
y_train = encoded_df['Premium Amount']  # Target variable

X_test = test_encoded_df.drop(columns=['id'])  # Drop 'id' from the test data

from sklearn.linear_model import LinearRegression

# Fit the Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict Premium Amount
y_pred = model.predict(X_test)

Next steps:

testing manually the model
download the file to upload to Kaggle

In [45]:
# Example manual input with the specified columns
manual_input = pd.DataFrame({
    'Age': [30],
    'Annual Income': [60000],
    'Number of Dependents': [1],
    'Education Level': [2],  # Assuming 2 = Master's degree
    'Health Score': [75],
    'Policy Type': [1],  # Assuming 1 = Comprehensive
    'Previous Claims': [0],
    'Vehicle Age': [3],
    'Credit Score': [720],
    'Insurance Duration': [10],
    'Exercise Frequency': [2],  # Assuming 2 = Weekly
    'Gender_Male': [1],  # 1 = Male, 0 = Female
    'Marital Status_Married': [1],  # 1 = Married, 0 = Not Married
    'Marital Status_Single': [0],  # 1 = Single, 0 = Not Single
    'Occupation_Self-Employed': [0],  # 1 = Self-Employed, 0 = Otherwise
    'Occupation_Unemployed': [0],  # 1 = Unemployed, 0 = Otherwise
    'Location_Suburban': [1],  # 1 = Suburban, 0 = Otherwise
    'Location_Urban': [0],  # 1 = Urban, 0 = Otherwise
    'Property Type_Condo': [1],  # 1 = Condo, 0 = Otherwise
    'Property Type_House': [0],  # 1 = House, 0 = Otherwise
    'Smoking Status_Yes': [0],  # 1 = Smoker, 0 = Non-Smoker
    'Policy Duration (days)': [365],  # 1 Year Policy Duration
    'Age_Health_Interaction': [30 * 75],  # Interaction term (Age * Health Score)
})

manual_input

,Age,Annual Income,Number of Dependents,Education Level,Health Score,Policy Type,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,...,Marital Status_Single,Occupation_Self-Employed,Occupation_Unemployed,Location_Suburban,Location_Urban,Property Type_Condo,Property Type_House,Smoking Status_Yes,Policy Duration (days),Age_Health_Interaction
0,30,60000,1,2,75,1,0,3,720,10,...,0,0,0,1,0,1,0,0,365,2250


In [46]:
from sklearn.preprocessing import StandardScaler

# Assuming the scaler was already fitted on encoded_df or X_train
# scaler = StandardScaler()
# scaler.fit(X_train)  # This step is already done during preprocessing

# Apply the scaling to the manual_input
manual_input[numerical_columns] = scaler.transform(manual_input[numerical_columns])

In [47]:
# Predict Premium Amount
manual_prediction = model.predict(manual_input)

print(f"Predicted Premium Amount: {manual_prediction[0]}")

Predicted Premium Amount: 2200.8478295748328


In [48]:
# Create a DataFrame for the submission file
submission = pd.DataFrame({
    'id': test_encoded_df['id'],  # ID column from the test set
    'Premium Amount': y_pred      # Predicted Premium Amount values
})


In [49]:
# Save the submission file as 'submission.csv'
submission.to_csv('submission.csv', index=False)